## Instrucciones

Edita, elabora y ejecuta los comandos y responde a las preguntas en este mismo documento. Pero **¡atención!** debes descargarlo (en formato `.ipynb`) para poder entregarlo a través del aula virtual. Es recomendable descargar de cuando en cuando versiones parciales, para evitar la pérdida eventual de todo el trabajo si la sesión se desconecta.

## Problema 1
En la carpeta `data` hay cuatro ficheros en formato FASTQ, con secuencias de cuatro muestras diferentes de cerbero común (*Canis cerberus*): `data/BRZ026.fq`, `data/BRZ031.fq`, `data/LAN048.fq` y `data/LAN098.fq`. El objetivo es: primero realizar un análisis de la calidad de las secuencias y después mapearlas sobre el genoma de referencia, `data/cerber.fa`. Pero los bloques de código siguientes están desordenados y contienen errores. Corrige los errores y ejecútalos en el orden correcto para poder responder las preguntas siguietes.

In [ ]:
# BLOQUE METBREWER

library('MetBrewer')
system2(command = 'cut', args = c('-f', '4,5', 'BRZ026.sam'), stdout = 'MapQualBRZ026.txt')
system3(command = 'cut', args = c('-f', '4,5', 'BRZ031.sam'), stdout = 'MapQualBRZ031.txt')
system2(command = 'cut', args = c('-f', '4,5', 'LAN048.sam'), stdout = 'MapQualLAN048.txt')
system2(command = 'cut', args =  ('-f', '4,5', 'LAN098.sam'), stdout = 'MapQualLAN098.txt')

mapq.BRZ026 <- read.table('MapQualBRZ026.txt', col.names = c('pos','mapq'))
mapq.BRZ031 <- read.table('MapQualBRZ031.txt', col.names = c('pos','mapq'))
mapq.LAN048 <- read.table('MapQualLAN048.txt', col.names = c('pos','mapq'))
mapq.LAN098 <- read.table('MapQualLAN098.txt', col.names = c('pos','mapq'))

colors = met.brewer(name = 'Egypt', n = 4)
plot(density(mapq.BRZ031$mapq),  lwd = 2, col = colors[1], xlab = 'Calidad del mapeo', ylab = 'Densidad', main = '')
lines(density(mapq.BRZ026$mapq), lwd = 2, col = colors[2])
lines(density(mapq.LAN048$mapq), lwd = 2, col = colors[3])
lines(density(mapq.LAN098$mapq), lwd = 2, col = colors[4])
legend(10, y=0.08, legend = c('BRZ031','BRZ026','LAN048','LAN098'), col = colors, lwd = 2)

In [ ]:
# BLOQUE BOWTIE
BRZ026.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ026.sam',
                          seq1 = 'data/BRZ026.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ026.out

BRZ031.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'BRZ031.sam',
                          seq1 = 'data/BRZ031.fa',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
BRZ031.out

LAN048.out < bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN048.sam',
                          seq1 = 'data/LAN048.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN048.out

LAN098.out <- bowtie2(bt2Index = 'index/cerber',
                     samOutput = 'LAN098.sam',
                          seq1 = 'data/LAN098.fq',
                     overwrite = TRUE,
                     '--no-unal', '--no-head')
LAN098.out

In [ ]:
# BLOQUE GGPLOT2

library('ggplot2')
suppressMessages(library('ShortRead'))
ResumenCalidad <- qa('data', type = 'fastq', pattern = '*.fasta')
ResumenCalidad
head(ResumenCalidad[['baseCalls']])
report(ResumenCalidad, dest = 'ResumenCalidad.html')

ggplot(data = ResumenCalidad[['readQualityScore']],
       mapping = aes(x = quality, y = density)) +
   geom_line() +
   facet_wrap(~lane)

In [ ]:
# BLOQUE RBOWTIE2

library('Rbowtie2')
dir.create('index')
bowtie2_build(references = 'data/cerber.fq',
              bt2Index   = 'indice/cerbero',
              '--quiet', overwrite = TRUE)

Contesta las preguntas siguientes:

### 1.1. Indica en qué orden crees que deben ejecutarse los pasos (puedes ordenar los bloques moviéndolos arriba y abajo).
### 1.2. ¿Cuántas lecturas cortas hay en cada archivo? (Si necesitas ejecutar algo para averiguarlo, añade un bloque de código).
### 1.3. ¿Crees que las lecturas cortas son emparejadas (*paired ends*) o no? ¿Cómo lo has averiguado?
### 1.4. ¿Qué longitudes tienen las lecturas?
### 1.5. Se sospecha que una de las tres muestras no pertenece a la misma especie que las otras. ¿Cuál dirías que és y por qué?

## Problema 2

En el archivo `data/reads.fasta` están las 1000 primeras lecturas cortas de la muestra LAN048, en formato FASTA. A continuación, buscamos esas 1000 lecturas mediante BLAST en el genoma, para replicar parte del trabajo ejecutado en el ejercicio anterior, ahora con BLAST. Corrige los errores donde los haya y contesta las preguntas siguientes.

In [1]:
# BLOQUE MAKEBLASTDB
system2(command = 'makeblastdb',
        args = c('-in', 'data/cerber.fa',
                 '-input_type', 'fasta',
                 'dbtype', 'nucl'))

In [2]:
# BLOQUE BLAST
system2(command = 'blastp',
        args = c('-db', 'data/cerber.fa',
                 '-query', 'data/reads.fasta',
                 '-evalue', '1.0E-20',
                 '-outfmt', '7',
                 '-out', 'blastout.txt'))

In [ ]:
# BLOQUE MULTIPLICIDAD
blastout <- read.table('blastout.txt',
                       col.names = c('query','subject','identity','length','mismatches',
                                     'gaps','qstart','qend','sstart','send','evalue',
                                     'bitscore'))
NumHits <- table(blastout$query)
Multiplicidad <- table(NumHits)
Multiplicidad

### 2.1 ¿Para qué sirve el primero de los dos bloques de código anteriores (el etiquetado como "BLOQUE MAKEBLASTDB")?

### 2.2 Si las mismas secuencias *query* las hubiéramos buscado en una base de datos más grande, con los genomas de muchas otras especies, ¿cómo hubieran sido los valores E de los resultados, iguales, mayores o menores? Razona la respuesta.

### 2.3 ¿Por qué motivo crees que algunas lecturas cortas encuentran más de una posición en el genoma con una identidad del 100%?

### 2.4 ¿Qué crees que representa el objeto `Multiplicidad`? (Puedes comprobar cuánto suman sus términos, o cómo cambian los números cuando reduces el parámetro `-evalue` en el *BLOQUE BLAST").


## Problema 3

El archivo `data/Kphage.fasta` contiene las 101 proteínas codificadas en el genoma del fago vB_KpnP_P184 de *Klebsiella*. El bloque de código siguiente debería crear una tabla con los resultados de buscar qué dominios proteicos de Pfam-A aparecen en esas 101 proteínas. Pero para que funcione hace falta un paso previo.

In [ ]:
# PASO PREVIO


In [ ]:
# BLOQUE HMMSCAN
## BLOQUE HMMSCAN
system2(command = 'hmmscan',
        args = c('--tblout', 'Kphage.tblout',
                 '--domtblout', 'Kphage.domtblout',
                 '-E', '0.00001',
                 '--domE', '0.00001',
                 '--noali', '-o', 'Kphage.out',
                 'data/Pfam.hmm', 'data/Kphage.fasta'))

### 3.1 Revisa los apuntes de la práctica del tema 5, sobre HMMER y Pfam, o bien consulta la ayuda del paquete HMMER en la carpeta de documentos si hace falta para completar el paso previo necesario para que funcione `hmmscan`.

### 3.2 ¿Qué diferencias encuentras entre los archivos de salida `Kphage.tblout` y `Kphage.domtblout`?

### 3.3 Realiza una segunda búsqueda de dominios proteicos ahora entre las 100 proteínas codificadas por otro fago (éste, de *Mycobacterium*), guardadas en formato FASTA en el archivo `data/Mphage.fasta`. (Esta búsqueda se puede realizar tanto con `hmmscan` como con `hmmsearch`. El programa `hmmsearch` funcionaría inclusio si no hubieras resuelto la pregunta 3.1

### El bloque siguiente leerá la tabla de resultados `Kphage.domtblout` en un *data frame*. Repite la operación para los resultados de la pregunta 3.3 y trata de averiguar qué dominios tienen en común los proteomas de los dos fagos.

In [1]:
library(rhmmer)
KphageDom <- read_domtblout('Kphage.domtblout')
names(KphageDom)
KphageDom
unique(KphageDom$domain_name)